In [8]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from names_dataset import NameDataset

In [2]:
file_path = "../data/train.json"

with open(file_path, "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [3]:
df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [26]:
labels = df.labels.explode().reset_index(drop=True)
labels_vc = labels.value_counts()
labels_vc

labels
O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [27]:
labels[labels == "B-ID_NUM"]

24543      B-ID_NUM
24556      B-ID_NUM
24567      B-ID_NUM
24573      B-ID_NUM
24579      B-ID_NUM
             ...   
1842369    B-ID_NUM
2840591    B-ID_NUM
2841190    B-ID_NUM
3889614    B-ID_NUM
4423827    B-ID_NUM
Name: labels, Length: 78, dtype: object

In [31]:
# we check how a B-ID_NUM looks like by getting the first one
idx = labels[labels == "B-ID_NUM"].index
print(idx)
df.tokens.explode().reset_index(drop=True)[idx]

Index([  24543,   24556,   24567,   24573,   24579,   24585,   24591,   24597,
         24606,   80831,   97507,   97514,  174586,  174597,  189006,  189037,
        189043,  193108,  193151,  193209,  196376,  196380,  201383,  239362,
        239366,  239606,  239610,  239908,  239912,  240133,  240137,  244287,
        254851,  254855,  301452,  301459,  309632,  309641,  309650,  311986,
        312422,  333222,  344705,  355363,  355380,  355397,  458156,  464653,
        604680,  604859,  715065,  715072,  727949,  728018,  751539,  751546,
        831454,  831672,  831691,  831758,  831847, 1086446, 1112207, 1613906,
       1614146, 1614236, 1614274, 1724741, 1725060, 1725124, 1725165, 1832446,
       1841585, 1842369, 2840591, 2841190, 3889614, 4423827],
      dtype='int64')


24543      860632713425
24556      530670102508
24567      530670102508
24573      875673967537
24579      860632713425
               ...     
1842369       047378465
2840591         IV-8322
2841190         IV-8322
3889614            Z.S.
4423827          V69230
Name: tokens, Length: 78, dtype: object

## Model Definition
We will attempt to create the simplest model possible. The model will evaluate each token and pass it through a couple functions to evaluate it. If one of the functions returns true, the token will be considered a keyword. The model will then return the keyword and the index of the token in the input string.

### Model Functions
- **email** This function will use regex to identify if the token is an email address.
- **phone_num** This function will use regex to identify if the token is a phone number.
- **address** This function will use regex to identify if the token is an address.
- **username** This function will use a word embedding model to identify if the token is a common word and if not then we flag it as a username.
- **personal_id** This function will use regex to identify if the token is a personal id.


In [ ]:
class PiiModel:
    def __init__(self, train_set: pd.DataFrame) -> None:
        self.train_set = train_set
        pass

    def get_name_dataset(self):
        return NameDataset()
    
    def get_email_regex(self):
        return r'[\w\.-]+@[\w\.-]+'
    
    def get_phone_number_regex(self):
        return r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'
    
    def get_address_regex(self):
        return r'\d{1,5}\s\w.\s(\b\w*\b\s){1,2}\w*\.'
    
    def get_personal_id_regex(self):
        # Any 
        return r'\b\d{3}-\d{2}-\d{4}\b'
    

